## ERA5 Wind Gust Retrieval & Visualization

In [ ]:
import cdsapi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import imageio.v2 as imageio 
import io
import xarray as xr
import os

In [ ]:
##### USER INPUTS
try:
    year = int(input("📅 Enter the year (YYYY): "))
    month = int(input("📅 Enter the month (MM): "))
    start_day = int(input("📅 Enter the start day (DD): "))
    end_day = int(input("📅 Enter the end day (DD): "))
except ValueError:
    raise ValueError("Year, month, and days must be valid numbers!")

# Bounding Box (Latitude Max, Longitude Min, Latitude Min, Longitude Max)
try:
    print("\n 🌍 Enter the bounding box coordinates:")
    lat_max = float(input("🔼 Latitude Max: "))
    lon_min = float(input("◀️ Longitude Min: "))
    lat_min = float(input("🔽 Latitude Min: "))
    lon_max = float(input("▶️ Longitude Max: "))
except ValueError:
    raise ValueError("Latitude and Longitude values must be valid numbers!")

# Validate date range
if start_day > end_day or start_day < 1 or end_day > 31:
    raise ValueError("Invalid date range! Ensure start_day ≤ end_day and within valid days.")

# Construct Filename
grib_filename = f"{start_day:02d}_to_{end_day:02d}_{month:02d}{year}_10m_wind_gust.grib"

# DOWNLOAD ERA5 DATA 
dataset = "reanalysis-era5-single-levels"
request = {
    "variable": ["instantaneous_10m_wind_gust"],  
    "product_type": ["reanalysis"],
    "year": [str(year)],  
    "month": [f"{month:02d}"],  
    "day": [f"{d:02d}" for d in range(start_day, end_day + 1)],  # Multiple days as a list
    "time": [f"{hour:02d}:00" for hour in range(24)],  
    "area": [lat_max, lon_min, lat_min, lon_max],  
    "data_format": "grib"
}

client = cdsapi.Client()
print(f"\n Downloading ERA5 Wind Gust Data from {start_day:02d}/{month:02d}/{year} to {end_day:02d}/{month:02d}/{year} :")
client.retrieve(dataset, request).download(grib_filename)

In [ ]:
# Load and process data
ds = xr.open_dataset(grib_filename, engine="cfgrib")

print("Variables present in the GRIB file:", ds)

In [ ]:
# Convert to DataFrame & filter NaNs
df = ds.to_dataframe().reset_index()
df_filtered = df.dropna(subset=["i10fg"]).copy()
df_filtered.loc[:, "valid_time"] = pd.to_datetime(df_filtered["valid_time"])
df_filtered.head()

In [ ]:
# Extract unique valid times
valid_times = df_filtered["valid_time"].unique()

# Construct GIF filename
gif_filename = os.path.splitext(grib_filename)[0] + ".gif"

# Generate GIF
print("\n Creating GIF visualization ...")
extent = [lon_min, lon_max, lat_min, lat_max]

with imageio.get_writer(gif_filename, mode="I", duration=4, loop=0) as writer:
    for i, vtime in enumerate(valid_times):
        fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
        ax.set_extent(extent, crs=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.BORDERS, linestyle=":")
        ax.add_feature(cfeature.LAND, edgecolor="black")

        df_time = df_filtered[df_filtered["valid_time"] == vtime]
        gust_data = df_time.pivot(index="latitude", columns="longitude", values="i10fg").sort_index(ascending=False)

        img = ax.pcolormesh(gust_data.columns, gust_data.index, gust_data.values, cmap="coolwarm", shading="auto", transform=ccrs.PlateCarree())
        cbar = plt.colorbar(img, ax=ax, orientation="vertical", label="10m Wind Gust (m/s)")

        ax.set_title(f"Wind Gust at 10m | {vtime.strftime('%Y-%m-%d %H:%M')}", fontsize=12)

        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=300)
        plt.close()
        
        buf.seek(0)
        image = imageio.imread(buf)
        writer.append_data(image)

print(f" GIF saved as: {gif_filename}")

In [ ]:
# Display GIF
from IPython.display import display, Image
display(Image(filename=gif_filename))

## Wind Storm Detection & Trajectory Mapping

In [ ]:
from geopy.distance import geodesic
import numpy as np
import pandas as pd
from scipy.ndimage import label, center_of_mass, find_objects
from shapely.geometry import LineString, Point, Polygon

In [ ]:
VENT_MIN = 24.7  # Seuil minimal de vitesse du vent (en m/s)
SURFACE_MIN = 50  # Seuil minimum de surface (en km²)

In [ ]:
# Compute spatial resolution
latitudes = ds.latitude.values
longitudes = ds.longitude.values

if len(latitudes) > 1 and len(longitudes) > 1:
    resolution_km = geodesic((latitudes[0], longitudes[0]), (latitudes[1], longitudes[0])).kilometers
    RESOLUTION_KM2 = resolution_km ** 2
else:
    RESOLUTION_KM2 = 10 

In [ ]:
from scipy.interpolate import splprep, splev

# Smooth storm trajectory using B-Spline interpolation
def smooth_trajectory(line, smoothing_factor=0.5):
    if len(line.coords) < 4:  # Must have at least 4 points for B-spline 
        print("Not enough points for smoothing, returning original trajectory.")
        return line  

    # Remove duplicate consecutive points
    unique_coords = []
    for i, coord in enumerate(line.coords):
        if i == 0 or coord != line.coords[i - 1]:  # Only keep unique consecutive points
            unique_coords.append(coord)

    if len(unique_coords) < 4:  # Ensure enough unique points for B-spline
        print("Not enough unique points for smoothing, returning original trajectory.")
        return LineString(unique_coords)

    # Convert to numpy arrays
    x, y = zip(*unique_coords)
    x, y = np.array(x), np.array(y)

    # Check for invalid values
    if np.any(np.isnan(x)) or np.any(np.isnan(y)) or np.any(np.isinf(x)) or np.any(np.isinf(y)):
        print("Invalid coordinates detected, returning original trajectory.")
        return line

    # Create a B-Spline curve
    try:
        tck, u = splprep([x, y], s=smoothing_factor, k=3)  # Ensure valid smoothing
        u_fine = np.linspace(0, 1, len(x) * 10)  # More points for a smooth curve
        x_smooth, y_smooth = splev(u_fine, tck)

        return LineString(zip(x_smooth, y_smooth))

    except Exception as e:
        print(f"Spline interpolation failed: {e}, returning original trajectory.")
        return line


# Detect high wind-speed areas, extract storm centroids, and track their movement over time
def detect_wind_clusters(ds):
    points = []  
    all_points = []  
    linestrings = []  
    surfaces = []  

    time_values = ds.time.values  
    step_values = ds.step.values

    for base_time in time_values:
        for step in step_values:
            step_timedelta = pd.to_timedelta(step)
            datatime = pd.to_datetime(base_time) + step_timedelta

            print(f"Processing time step: {datatime}")

            # Extract wind speed data
            wind_speed = ds["i10fg"].sel(time=base_time, step=step).values
            mask = wind_speed >= VENT_MIN
            labeled_array, num_features = label(mask)

            # Extract storm regions
            regions = find_objects(labeled_array)

            max_area = 0
            best_centroid = None

            for i, region_slice in enumerate(regions, start=1):
                if region_slice is None:
                    continue  

                region = labeled_array[region_slice] == i
                area = np.sum(region) * RESOLUTION_KM2

                if area >= SURFACE_MIN:
                    y_start, y_end = region_slice[0].start, region_slice[0].stop
                    x_start, x_end = region_slice[1].start, region_slice[1].stop

                    coords = [(longitudes[x], latitudes[y]) 
                              for y in range(y_start, y_end) 
                              for x in range(x_start, x_end) if region[y - y_start, x - x_start]]

                    coords = list(set(coords))  
                    coords.append(coords[0])  

                    if len(coords) >= 4:
                        polygon = Polygon(coords)
                        surfaces.append({"datetime": datatime, "geometry": polygon, "surface_area": area})

                    # Compute centroid for the largest storm area
                    if area > max_area:
                        y, x = center_of_mass(region)
                        best_centroid = Point(longitudes[int(x_start + x)], latitudes[int(y_start + y)])
                        max_area = area

            if best_centroid:
                if len(points) > 0:
                    last_point = points[-1]

                    # Compute distance between last point and new centroid
                    distance = geodesic((last_point["geometry"].y, last_point["geometry"].x), 
                                        (best_centroid.y, best_centroid.x)).kilometers

                    if distance > 300:  # Threshold for separating trajectories
                        print(f"Storm shift detected ({distance:.2f} km), starting new trajectory.")

                        # Save previous storm's centroids before resetting points
                        all_points.extend(points)  

                        # Save previous trajectory if it has more than one point and apply smoothing
                        if len(points) > 1:
                            sorted_points = sorted(points, key=lambda p: p["datetime"])
                            raw_line = LineString([p["geometry"] for p in sorted_points])
                            smoothed_line = smooth_trajectory(raw_line, smoothing_factor=0.01) 
                            linestrings.append({"geometry": smoothed_line})

                        # Start a new trajectory
                        points = []

                # Append centroid to trajectory without filtering duplicates
                points.append({"datetime": datatime, "geometry": best_centroid})
                print(f" Centroid recorded at {datatime}: {best_centroid}, Area: {max_area:.2f} km²")

    # Save last storm's centroids
    all_points.extend(points)

    # Save the last trajectory
    if len(points) > 1:
        sorted_points = sorted(points, key=lambda p: p["datetime"])
        raw_line = LineString([p["geometry"] for p in sorted_points])
        smoothed_line = smooth_trajectory(raw_line, smoothing_factor=0.01)  
        linestrings.append({"geometry": smoothed_line})

    return all_points, linestrings, surfaces  

# Run storm detection
points, lines, surfaces = detect_wind_clusters(ds)

In [ ]:
import geopandas as gpd
import os

def export_to_shapefile(data, output_path):
    if not data:
        print(f"No data available for export: {output_path}")
        return

    # Convert Data to GeoDataFrame
    gdf = gpd.GeoDataFrame(data, crs="EPSG:4326")

    # Convert datetime column to string format if it exists
    if "datetime" in gdf.columns:
        gdf["datetime"] = gdf["datetime"].astype(str)  

    # Rename 'surface_area' to 'area' if present
    if "surface_area" in gdf.columns:
        gdf = gdf.rename(columns={"surface_area": "area"})  

    # Export to shapefile
    gdf.to_file(output_path)
    print(f"Export successful: {output_path}")

# Use the actual GRIB filename for naming
file_name = os.path.splitext(os.path.basename(grib_filename))[0]
wind_threshold = f"_V{VENT_MIN}"
area_threshold = f"_S{SURFACE_MIN}"

# Export shapefiles
export_to_shapefile(points, f"WC_{file_name}{wind_threshold}{area_threshold}.shp")  # Wind Centroids
export_to_shapefile(lines, f"WT_{file_name}{wind_threshold}{area_threshold}.shp")  # Wind Trajectories
export_to_shapefile(surfaces, f"WS_{file_name}{wind_threshold}{area_threshold}.shp")  # Wind Surfaces


## Interactive Map Visualization

In [ ]:
from ipyleaflet import Map, Marker, Popup, Polyline, Polygon, LayerGroup, TileLayer, basemaps, basemap_to_tiles, LayersControl, CircleMarker
from ipywidgets import HTML

In [ ]:
# Create the map
center = [(lat_min + lat_max) / 2, (lon_min + lon_max) / 2]
m = Map(center=center, zoom=6, basemap=basemap_to_tiles(basemaps.Esri.WorldImagery))

# Define layers
trajectory_layer = LayerGroup(name="Trajectories")
surface_layer = LayerGroup(name="Surfaces")
centroid_layer = LayerGroup(name="Centroids")

# Add trajectories
for line in lines:
    polyline = Polyline(
        locations=[(coord[1], coord[0]) for coord in line["geometry"].coords], 
        color="red",
        weight=2,  
        opacity=1.0,  
        fill=False,   
        name="Trajectory",
    )
    trajectory_layer.add_layer(polyline)

# Add surfaces
for surface in surfaces:
    polygon = Polygon(
        locations=[(coord[1], coord[0]) for coord in surface['geometry'].exterior.coords],
        color="blue",
        fill_color="blue",
        weight=1,
        opacity=0.5,
        name="Surface"
    )
    surface_layer.add_layer(polygon)

# Add centroids
for point in points:
    location = (point['geometry'].y, point['geometry'].x)
    
    # Create a Circlemarker for each centroid
    marker = CircleMarker(
        location=location,
        radius=1,
        color="yellow",
        fill_color="yellow",
        fill_opacity=1,
        draggable=False
    )
    
    popup_content = HTML(f"<b>Datetime:</b> {point['datetime']}<br><b>Location:</b> {location}")
    
    marker.popup = popup_content
    
    centroid_layer.add_layer(marker)

# Add layer groups to the map
m.add_layer(trajectory_layer)
m.add_layer(surface_layer)
m.add_layer(centroid_layer)

# Add layer control
layer_control = LayersControl(position="topright")
m.add_control(layer_control)

# Display the map
display(m)
